In [ ]:
import matplotlib.pyplot as plt
import numpy as np
!pip uninstall tensorflow
!pip install tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy, AUC, Accuracy

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

#Define information about training images 

training_generator = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.2,
        rotation_range=15,
        width_shift_range=0.05,
        height_shift_range=0.05
)
# Imported training images with specific requirements
training_iterator = training_generator.flow_from_directory(
  'Covid19-dataset/train',
  class_mode = "categorical",
  color_mode = 'grayscale',
  target_size = (256,256),
  batch_size = 32,
  shuffle=True
)

# Define information about validation images 
validation_generator = ImageDataGenerator(rescale=1./255)
# Imported training images with specific requirements
validation_iterator = validation_generator.flow_from_directory(
  'Covid19-dataset/test',
  class_mode = "categorical",
  color_mode = 'grayscale',
  target_size = (256,256),
  batch_size = 32,
  shuffle=True
)


In [ ]:
# Get the first batch of images and labels from the iterator
images, labels = training_iterator.next()

# Create a figure with 10 subplots
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(10, 5))

# Display the first 10 images in the batch
for i, ax in enumerate(axes.flatten()):
    ax.imshow(images[i], cmap='gray')
    ax.set_title(f"Label: {labels[i]}")
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# plotting categorical and validation accuracy over epochs
es = EarlyStopping(monitor='categorical_accuracy', mode='max', verbose=1, patience=10)

def fit_model(model_type, opt):
    # compiling the model
    model_type.compile(
        optimizer=opt,
        loss='categorical_crossentropy',
        metrics=['CategoricalAccuracy','AUC']
    )
    # training the model
    history = model_type.fit(
        training_iterator,
        steps_per_epoch=len(training_iterator),
        epochs=20,
        validation_data=validation_iterator,
        validation_steps=len(validation_iterator),
        callbacks=[es]
    )

    return history

def plotting_accuracy(history):
    # plotting categorical and validation accuracy over epochs
    fig = plt.figure()
    ax1 = fig.add_subplot(2, 1, 1)
    ax1.plot(history.history['categorical_accuracy'])
    ax1.plot(history.history['val_categorical_accuracy'])
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.grid()
    ax1.legend(['train', 'validation'], loc='upper left')

    # plotting auc and validation auc over epochs
    ax2 = fig.add_subplot(2, 1, 2)
    ax2.plot(history.history['auc'])
    ax2.plot(history.history['val_auc'])
    ax2.set_title('Model AUC')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('AUC')
    ax2.grid()
    ax2.legend(['train', 'validation'], loc='upper left')

    # used to keep plots from overlapping
    fig.tight_layout()

def evaluation_metrics(model_type):

    y_pred = model_2.predict(validation_iterator)
    y_pred = np.argmax(y_pred, axis=1)
    true_classes = validation_iterator.classes
    # class_labels = list(validation_iterator.class_indices.keys())
    
    cm = confusion_matrix(true_classes, y_pred)
    print('Confusion Matrix')
    print(cm)
    print('Classification Report')
    print(classification_report(true_classes, y_pred))

In [ ]:
# Create Model
model_1 = Sequential()

# Input Layer
model_1.add(Input(shape=(256,256,1)))

# Flatten Layer
model_1.add(Flatten())
# Output Layer
model_1.add(Dense(3, activation='softmax'))

# Summary report 
model_1.summary()

In [ ]:
opt = Adam(learning_rate=0.01)

model_1.compile(
        optimizer=opt,
        loss='categorical_crossentropy',
        metrics=['CategoricalAccuracy','AUC'],
)

# training the model
history = model_1.fit(
    training_iterator,
    steps_per_epoch=len(training_iterator),
    epochs=10,
    validation_data=validation_iterator,
    validation_steps=len(validation_iterator),
    callbacks=[es]
)
plotting_accuracy(history_1)
evaluation_metrics(model_1)

In [ ]:
# Create Model
model_2 = Sequential()

# Input Layer
model_2.add(Input(shape=(256,256,1)))

# Hidden Layer
model_2.add(Conv2D(32,3,activation='relu'))
model_2.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model_2.add(Conv2D(32,3,activation='relu'))
model_2.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model_2.add(Conv2D(32,3,strides=3,activation='relu'))
model_2.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# Flatten Layer
model_2.add(Flatten())

# Dropout Layer
model_2.add(Dropout(0.02))

# Output Layer
model_2.add(Dense(3, activation='softmax'))

# Summary report 
model_2.summary()

In [ ]:
history_2 = fit_model(model_2, Adam(learning_rate=0.01))
plotting_accuracy(history_2)

In [ ]:
# Create Model
model_3 = Sequential()

# Input Layer
model_3.add(Input(shape=(256,256,1)))

# Hidden Layer
model_3 = Sequential()
# add input layer with grayscale image shape
model_3.add(Input(shape=(256, 256, 1)))
# convolutional hidden layers with relu functions
# maxpooling layers and dropout layers as well
model_3.add(Conv2D(5, 5, strides=2, activation="relu")) 
model_3.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model_3.add(Dropout(0.01))
model_3.add(Conv2D(3, 3, strides=1, activation="relu")) 
model_3.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model_3.add(Dropout(0.02))

# Flatten Layer
model_3.add(Flatten())

# Dropout Layer
model_3.add(Dropout(0.01))

# Output Layer
model_3.add(Dense(3, activation='softmax'))

# Summary report 
model_3.summary()

In [ ]:
# Compile optimizer, loss and metrics to enhance the model performance
history_3 = fit_model(model_3, Adam(learning_rate=0.001))
plotting_accuracy(history_3)
evaluation_metrics(model_3)